# XOR Multiple Inputs/Targets

Testing...

In [1]:
from conx import Network, Layer, SGD

Using Theano backend.


In [2]:
net = Network("XOR", 2, 5, 1, activation="sigmoid")
net.connect()
dataset = [
    ([0, 0], [0]),
    ([0, 1], [1]),
    ([1, 0], [1]),
    ([1, 1], [0])
]

In [3]:
net.set_dataset(dataset)

Split dataset into:
   train set count: 4
   test set count : 0
Input Summary:
   count  : 4
   shape  : (2,)
   range  : (0.0, 1.0)
Target Summary:
   count  : 4
   shape  : (1,)
   range  : (0.0, 1.0)


In [4]:
net.get_target(0)

[0.0]

In [5]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [6]:
#net.config["image_maxdim"] = 50

In [7]:
net.dashboard()

<IPython.core.display.Javascript object>

In [8]:
net.propagate([-1, 1])

[0.40329412]

In [9]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch #   25 | train error 0.24345 | train accuracy 0.75000 | validate% 0.00000
Epoch #   50 | train error 0.22327 | train accuracy 0.75000 | validate% 0.00000
Epoch #   75 | train error 0.19210 | train accuracy 0.75000 | validate% 0.00000
Epoch #  100 | train error 0.16768 | train accuracy 0.75000 | validate% 0.00000
Epoch #  125 | train error 0.14064 | train accuracy 0.75000 | validate% 0.00000
Epoch #  150 | train error 0.09144 | train accuracy 1.00000 | validate% 0.00000
Epoch #  175 | train error 0.04804 | train accuracy 1.00000 | validate% 0.00000
Epoch #  200 | train error 0.02704 | train accuracy 1.00000 | validate% 0.00000
Epoch #  225 | train error 0.01702 | train accuracy 1.00000 | validate% 0.00000
Epoch #  250 | train error 0.01167 | train accuracy 1.00000 | validate% 0.50000
Epoch #  275 | train error 0.00853 | train accuracy 1.00000 | validate% 0.50000
Epoch #  289 | train error 0.00732 | train accuracy 1.00000 | validate% 1.00000


In [10]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0] | [0.0] | [0.1] | correct
1 | [0.0, 1.0] | [1.0] | [0.9] | correct
2 | [1.0, 0.0] | [1.0] | [0.9] | correct
3 | [1.0, 1.0] | [0.0] | [0.1] | correct
Total count: 4
Total percentage correct: 1.0


In [11]:
net.propagate_to("input", [0, 1])

[0.0, 1.0]

In [12]:
net.propagate([0.5, 0.5])

[0.88367403]

In [13]:
net.propagate_to("hidden", [1, 0])

[0.19145873188972473,
 0.0193781815469265,
 0.38365882635116577,
 0.368780255317688,
 0.6853697299957275]

In [14]:
net.propagate_to("output", [0.25, 0.75])

[0.8821930885314941]

In [15]:
net.propagate_to("input", [0.25, 0.25])

[0.25, 0.25]

In [16]:
net.propagate_from("input", [1.0, 1.0])

[0.8975997]

In [17]:
net.propagate_from("hidden", [1.0, 0.0, 1.0, 0.5, -1.0])

[0.67375487]

In [18]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [0.0, 0.0] | [0.0] | [0.1] | correct
1 | [0.0, 1.0] | [1.0] | [0.9] | correct
2 | [1.0, 0.0] | [1.0] | [0.9] | correct
3 | [1.0, 1.0] | [0.0] | [0.1] | correct
Total count: 4
Total percentage correct: 1.0


In [19]:
from conx import Network, Layer, SGD

In [25]:
net = Network("XOR2")
net.add(Layer("input1", shape=1))
net.add(Layer("input2", shape=1))
net.add(Layer("input3", shape=1))
net.add(Layer("hidden1", shape=10, activation="sigmoid"))
net.add(Layer("hidden2", shape=10, activation="sigmoid"))
net.add(Layer("shared-hidden", shape=5, activation="sigmoid"))
net.add(Layer("output1", shape=2, activation="sigmoid"))
net.add(Layer("output2", shape=2, activation="sigmoid"))
net.add(Layer("output3", shape=4, activation="sigmoid"))

In [21]:
net = Network("XOR2")
net.add(Layer("input1", shape=1))
net.add(Layer("input2", shape=1))
net.add(Layer("hidden1", shape=10, activation="sigmoid"))
net.add(Layer("hidden2", shape=10, activation="sigmoid"))
net.add(Layer("shared-hidden", shape=5, activation="sigmoid"))
net.add(Layer("output1", shape=1, activation="sigmoid"))
net.add(Layer("output2", shape=1, activation="sigmoid"))

In [22]:
net = Network("XOR2")
net.add(Layer("input1", shape=1, image_maxdim=100))
net.add(Layer("input2", shape=1, image_maxdim=50))
net.add(Layer("hidden1", shape=10, activation="sigmoid"))
net.add(Layer("hidden2", shape=10, activation="sigmoid"))
net.add(Layer("shared-hidden", shape=5, activation="sigmoid"))
net.add(Layer("output1", shape=1, activation="sigmoid", image_maxdim=100))
net.add(Layer("output2", shape=1, activation="sigmoid", image_maxdim=50))

In [26]:
net

<Network name='XOR2' (uncompiled)>

In [27]:
net.connect("input1", "hidden1")
net.connect("input2", "hidden2")
net.connect("input3", "hidden2")
net.connect("hidden1", "shared-hidden")
net.connect("hidden2", "shared-hidden")
net.connect("shared-hidden", "output1")
net.connect("shared-hidden", "output2")
net.connect("shared-hidden", "output3")

In [28]:
net.set_input_layer_order("input1", "input2", "input3")
net.set_output_layer_order("output1", "output2", "output3")

In [29]:
net.compile(loss='mean_squared_error',
            optimizer=SGD(lr=0.3, momentum=0.9))

In [30]:
#net.config["image_maxdim"] = 50
net.dashboard()

<IPython.core.display.Javascript object>

In [59]:
dataset = [
    ([[0],[0]], [[0],[0]]),
    ([[0],[1]], [[1],[1]]),
    ([[1],[0]], [[1],[1]]),
    ([[1],[1]], [[0],[0]])
]

In [60]:
net.set_dataset(dataset)

Split dataset into:
   train set count: 4
   test set count : 0
Input Summary:
   count  : 4
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   count  : 4
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [27]:
net.config["image_maxdim"] = 50

In [28]:
net.dashboard()

<IPython.core.display.Javascript object>

In [29]:
net["shared-hidden"].colormap = "spring"
net.propagate_from("hidden1", [0.0, 1.0] * 5)

[[0.69199979], [0.3493453]]

In [30]:
net["shared-hidden"].colormap = "hot"
net.propagate_from("hidden1", [1.0, 1.0] * 5, ["shared-hidden"])

[0.26171306, 0.76274085, 0.265212, 0.11297761, 0.66586345]

In [31]:
net.propagate_to("output1", [[1], [0]])

[0.5030538439750671]

In [32]:
for i in range(20):
    (epoch_count, loss, acc, val_acc) = net.train(epochs=100, verbose=0)
    for index in range(4):
        net.propagate(dataset[index][0])

In [33]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch # 2001 | train error 0.00151 | train accuracy 2.00000 | validate% 1.00000


In [34]:
net.propagate_from("input1", [0.0])

[[0.5], [0.5]]

In [35]:
net.propagate_from("shared-hidden", [0.0] * 5)

[[0.5], [0.5]]

In [36]:
net.propagate_to("hidden1", [[1], [1]])

[0.16668787598609924,
 0.13967588543891907,
 0.8931041955947876,
 0.8223327398300171,
 0.19805079698562622,
 0.7693232297897339,
 0.6709631681442261,
 0.8438820838928223,
 0.6473685503005981,
 0.12924261391162872]

In [37]:
net.test()

Testing on training dataset...
# | inputs | targets | outputs | result
---------------------------------------
0 | [[0.0], [0.0]] | [ 0.  0.] | [ 0.02752927  0.0273406 ] | correct
1 | [[0.0], [1.0]] | [ 1.  1.] | [ 0.96860015  0.97040695] | correct
2 | [[1.0], [0.0]] | [ 1.  1.] | [ 0.97567117  0.97366613] | correct
3 | [[1.0], [1.0]] | [ 0.  0.] | [ 0.0262641   0.02651409] | correct
Total count: 4
Total percentage correct: 1.0


In [38]:
net.slice_dataset(2)

Slicing dataset 0:2...
Input Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)
Target Summary:
   length  : 2
   training: 2
   testing : 0
   shape  : [(1,), (1,)]
   range  : (0.0, 1.0)


In [39]:
net.train(epochs=2000, accuracy=1.0, report_rate=25)

Training...
Epoch # 2002 | train error 0.00168 | train accuracy 2.00000 | validate% 1.00000


# Conx model is a Keras Model

In [40]:
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML

In [41]:
dot = model_to_dot(net.model, rankdir="BT")

In [42]:
HTML(dot.create_svg().decode())